# Interpretability With Tensorflow On Azure Machine Learning Service

## Overview of Tutorial
This notebook is Part 4 (Explaining Your Model Using Interpretability) of a four part workshop that demonstrates an end-to-end workflow for using Tensorflow on Azure Machine Learning Service. The different components of the workshop are as follows:

- Part 1: [Preparing Data and Model Training](https://github.com/microsoft/bert-stack-overflow/blob/master/1-Training/AzureServiceClassifier_Training.ipynb)
- Part 2: [Inferencing and Deploying a Model](https://github.com/microsoft/bert-stack-overflow/blob/master/2-Inferencing/AzureServiceClassifier_Inferencing.ipynb)
- Part 3: [Setting Up a Pipeline Using MLOps](https://github.com/microsoft/bert-stack-overflow/tree/master/3-ML-Ops)
- Part 4: [Explaining Your Model Interpretability](https://github.com/microsoft/bert-stack-overflow/blob/master/4-Interpretibility/IBMEmployeeAttritionClassifier_Interpretability.ipynb)

**In this specific tutorial, we will cover the following topics:**

- TODO
- TODO

## What is Azure Machine Learning Service?
Azure Machine Learning service is a cloud service that you can use to develop and deploy machine learning models. Using Azure Machine Learning service, you can track your models as you build, train, deploy, and manage them, all at the broad scale that the cloud provides.
![](./images/aml-overview.png)


## What Is Machine Learning Interpretability?
Interpretability is the ability to explain why your model made the predictions it did. The Azure Machine Learning service offers various interpretability features to help accomplish this task. These features include:

- Feature importance values for both raw and engineered features.
- Interpretability on real-world datasets at scale, during training and inference.
- Interactive visualizations to aid you in the discovery of patterns in data and explanations at training time.

By accurately interpretabiliting your model, it allows you to:

- Use the insights for debugging your model.
- Validate model behavior matches their objectives.
- Check for for bias in the model.
- Build trust in your customers and stakeholders.

![](./images/interpretability-architecture.png)

## Install Azure Machine Learning Python SDK

If you are running this on a Notebook VM, the Azure Machine Learning Python SDK is installed by default. If you are running this locally, you can follow these [instructions](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py) to install it using pip.

This tutorial series requires version 1.0.69 or higher. We can import the Python SDK to ensure it has been properly installed:

In [ ]:
import azureml.core

print("Azure Machine Learning Python SDK version:", azureml.core.VERSION)

## Install Tensorflow 1.14

We will be using an older version (1.14) for this particular tutorial in the series as Tensorflow 2.0 is not yet supported for Interpretibility on Azure Machine Learning service. If are currently running Tensorflow 2.0, run the code below to downgrade the version.

In [ ]:
%pip uninstall tensorflow-gpu
%pip install tensorflow-gpu==1.14

Let's make sure we have the right verison.

In [6]:
import tensorflow as tf
tf.version.VERSION

'2.0.0'

## Connect To Workspace

Just like in the previous tutorials, we will need to connect to a [workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace(class)?view=azure-ml-py).

The following code will allow you to create a workspace if you don't already have one created. You must have an Azure subscription to create a workspace:

```python
from azureml.core import Workspace
ws = Workspace.create(name='myworkspace',
                      subscription_id='<azure-subscription-id>',
                      resource_group='myresourcegroup',
                      create_resource_group=True,
                      location='eastus2')
```

**If you are running this on a Notebook VM, you can import the existing workspace.**

In [ ]:
from azureml.core import Workspace

workspace = Workspace.from_config()
print('Workspace name: ' + workspace.name, 
      'Azure region: ' + workspace.location, 
      'Subscription id: ' + workspace.subscription_id, 
      'Resource group: ' + workspace.resource_group, sep = '\n')

> **Note:** that the above commands reads a config.json file that exists by default within the Notebook VM. If you are running this locally or want to use a different workspace, you must add a config file to your project directory. The config file should have the following schema:

```
    {
        "subscription_id": "<SUBSCRIPTION-ID>",
        "resource_group": "<RESOURCE-GROUP>",
        "workspace_name": "<WORKSPACE-NAME>"
    }
```

## Write Training Script
For this tutorial, we will be using the *tf.keras module* to train a basic feed forward neural network on the IBM Employee Attrition Dataset. 

**We will start by writing the training cript into a train.py file**

In [44]:
%%writefile train.py
import pandas as pd 
import numpy as np
import tensorflow as tf
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

def preprocess_data(data):
    '''
    
    '''
    # Dropping Employee count as all values are 1 and hence attrition is independent of this feature
    data = data.drop(['EmployeeCount'], axis=1)
    
    # Dropping Employee Number since it is merely an identifier
    data = data.drop(['EmployeeNumber'], axis=1)
    data = data.drop(['Over18'], axis=1)

    # Since all values are 80
    data = data.drop(['StandardHours'], axis=1)

    # Converting target variables from string to numerical values
    target_map = {'Yes': 1, 'No': 0}
    data["Attrition_numerical"] = data["Attrition"].apply(lambda x: target_map[x])
    target = data["Attrition_numerical"]

    data.drop(['Attrition_numerical', 'Attrition'], axis=1, inplace=True)
    
    # Creating dummy columns for each categorical feature
    categorical = []
    for col, value in data.iteritems():
        if value.dtype == 'object':
            categorical.append(col)

    # Store the numerical columns in a list numerical
    numerical = data.columns.difference(categorical)   

    # We create the preprocessing pipelines for both numeric and categorical data.
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocess = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical),
            ('cat', categorical_transformer, categorical)])
    
    pipeline = make_pipeline(preprocess)

    # Split data into train and test sets
    x_train, x_test, y_train, y_test = train_test_split(data, 
                                                        target, 
                                                        test_size=0.2,
                                                        random_state=0,
                                                        stratify=target)
    
    # Transform data
    x_train_t = pd.DataFrame(pipeline.fit_transform(x_train))
    x_test_t = pd.DataFrame(pipeline.transform(x_test))
    
    return x_train_t, x_test_t, y_train, y_test
    
# Load and preprocess data
attrition_data = pd.read_csv('./data/data.csv')
x_train, x_test, y_train, y_test = preprocess_data(attrition_data)

# Create model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=16, activation='relu', input_shape=(x_train.shape[1],)))
model.add(tf.keras.layers.Dense(units=16, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy']) 

# Fit model
model.fit(x_train, y_train, epochs=20, verbose=1, batch_size=128, validation_data=(x_test, y_test))

# Save model
model.save('model.h5')

Writing train.py


## Explain Model Locally

We will start by training our model and explaining it in your local environment.

**First step is to run the training script that we just wrote**

In [1]:
import sys
!{sys.executable} train.py

2019-10-27 15:58:36.700862: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-10-27 15:58:36.723711: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2112000000 Hz
2019-10-27 15:58:36.728256: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fffd94ac210 executing computations on platform Host. Devices:
2019-10-27 15:58:36.728766: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
Train on 1176 samples, validate on 294 samples
Epoch 1/20
1176/1176 [==============================] - 0s 357us/sample - loss: 0.8064 - accuracy: 0.2959 - val_loss: 0.7107 - val_accuracy: 0.4898
Epoch 2/20
1176/1176 [==============================] - 0s 32us/sample - loss: 0.6746 - accuracy: 0.5561 - val_loss: 0.6247 - val_accuracy: 0.6837
Epoch 3/20
1176/1176 [==============================] - 0s 30us/sample - loss: 0.5971

**Load model and perform interpretability**

In [13]:
# TODO:  LOAD MODEL AND EXPLAIN IT
import tensorflow as tf

model = tf.keras.models.load_model('model.h5')

# from azureml.explain.model.tabular_explainer import TabularExplainer
# # "features" and "classes" fields are optional
# explainer = TabularExplainer(network, 
#                              train)

# # you can use the training data or the test data here
# global_explanation = explainer.explain_global(x_train)

# # if you used the PFIExplainer in the previous step, use the next line of code instead
# # global_explanation = explainer.explain_global(x_train, true_labels=y_test)

# # sorted feature importance values and feature names
# sorted_global_importance_values = global_explanation.get_ranked_global_values()
# sorted_global_importance_names = global_explanation.get_ranked_global_names()
# dict(zip(sorted_global_importance_names, sorted_global_importance_values))

# # alternatively, you can print out a dictionary that holds the top K feature names and values
# global_explanation.get_feature_importance_dict()

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


#### Train and Explain Locally
We will start by training our model locally in the Jupyter Notebook.

#### Train and Explain Remotely
Now we will train our model on the compute target created back in the [first tutorial]().

## Interpretability In Inferencing

## Raw Feature Transformations

## Visualizations